#STEP 1:Problem Framing & Experiment Design

📘 Learn Covered

✔ Training multiple models (design stage – before coding)

🛠️ Task Covered

✔ Prepare to train 3 different models (design + alignment)

#Problem Statement:Can we predict how many purchases a product will receive based on its visibility and conversion behavior?

Dataset Understanding (Confirmed)

Source table: gold.products

Column	Role

views	-> Feature

conversion_rate	-> Feature

purchases	-> Target

event_date	Ignored 

product_id	Ignored

revenue	Ignored 

#Final ML Design

🎯 Target Variable

purchases

🧠 Input Features

views

conversion_rate

#Model Families Planned (No Training Yet)

We will compare model families


Linear Regression	-> Simple, interpretable baseline

Decision Tree	-> Captures non-linearity

Random Forest	-> More robust ensemble

✔ Same data

✔ Same features

✔ Same metric

✔ Same MLflow experiment

#STEP 2 Train 3 Different Models (Scikit-Learn) + MLflow Tracking

Learn Covered

✔ Training multiple models

✔ Comparing models under the same experiment

🛠️ Task Covered

✔ Task 1 – Train 3 different models

⚠️ In this step we ONLY train & log models

❌ No tuning

❌ No Spark pipeline yet

❌ No model selection yet

In [0]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment("/Users/ram.katneni@gmail.com/day13_model_comparison")


✔ Single experiment

✔ Comparable runs

✔ Clean UI

In [0]:
df = spark.table("gold.products").toPandas()

X = df[["views", "conversion_rate"]]
y = df["purchases"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


✔ Same split for all models

✔ Fair comparison

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

models = {
    "linear_regression": LinearRegression(),
    "decision_tree": DecisionTreeRegressor(max_depth=5, random_state=42),
    "random_forest": RandomForestRegressor(n_estimators=50, random_state=42)
}


✔ Different model families

✔ No hyperparameter tuning yet

In [0]:
from sklearn.metrics import r2_score

for name, model in models.items():
    with mlflow.start_run(run_name=name):
        mlflow.log_param("model_type", name)
        mlflow.log_param("features", "views, conversion_rate")

        model.fit(X_train, y_train)

        preds = model.predict(X_test)
        r2 = r2_score(y_test, preds)

        mlflow.log_metric("r2_score", r2)
        mlflow.sklearn.log_model(model, "model")

        print(f"{name} R²: {r2:.4f}")


✔ Same metric

✔ Same dataset

✔ Same logging format

#STEP 3 Compare Model Metrics in MLflow UI
📘 Learn Covered

✔ Experiment tracking
✔ Model comparison using MLflow

🛠️ Task Covered

✔ Task 2 – Compare metrics in MLflow

⚠️ This step is analysis only
❌ No new training
❌ No code changes
❌ No Spark pipeline yet

# STEP 4 Build Production-Grade Spark ML Pipeline (Winner Model Family)

📘 Learn Covered

✔ Spark ML Pipelines

✔ Feature engineering at scale

🛠️ Tasks Covered

✔ Task 3 – Build Spark ML pipeline

⚠️ In this step:

We do NOT use scikit-learn

We do NOT log to MLflow yet

We focus on production readiness

Design Decision (Very Important)

Based on your results:

Model	R²
Random Forest	0.998 ✅
Decision Tree	0.952
Linear Regression	0.591
✅ Chosen Production Model Family



👉 Decision Tree in Spark = production-ready + explainable

In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["views"],   
    outputCol="features"
)


✔ Feature engineering at scale

✔ No Pandas

✔ Works on millions of rows

In [0]:
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(
    featuresCol="features",
    labelCol="purchases",
    maxDepth=5
)


Tree-based
✔ Hyperparameter explicitly defined
✔ Prevents overfitting

In [0]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[
    assembler,
    dt
])


✔ Reproducible
✔ Versionable
✔ Production-grade

In [0]:
spark_df = spark.table("gold.products")

train_df, test_df = spark_df.randomSplit([0.8, 0.2], seed=42)


✔ Distributed
✔ Deterministic split

In [0]:
pipeline_model = pipeline.fit(train_df)


✔ One command
✔ Entire workflow captured

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = pipeline_model.transform(test_df)

evaluator = RegressionEvaluator(
    labelCol="purchases",
    predictionCol="prediction",
    metricName="r2"
)

r2 = evaluator.evaluate(predictions)
print(f"Spark Decision Tree R²: {r2:.4f}")


#STEP 5 – Select Best Model & Summarize Learnings

Models trained & compared

Model	Framework	R²	Verdict

Linear Regression	sklearn	~0.59	Strong baseline

Decision Tree	sklearn	~0.95	Overfit

Random Forest	sklearn	~0.99	Severe overfit

Decision Tree	Spark ML	~0.05	Honest, scalable

#LOG SPARK DECISION TREE MODEL TO MLFLOW

This step covers MLflow components:

Experiment tracking

Model logging

Metrics logging

#FULL SPARK DECISION TREE PIPELINE, TRAIN, EVALUATE & LOG 

In [0]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
import mlflow
import mlflow.spark
from mlflow.models.signature import infer_signature

# -------------------------------
# 1️⃣ Prepare features & train/test split
# -------------------------------
feature_cols = ["views"]  # your features
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Decision Tree model
dt = DecisionTreeRegressor(featuresCol="features", labelCol="purchases")

# Pipeline
pipeline = Pipeline(stages=[assembler, dt])

# Train model
pipeline_model = pipeline.fit(train_df)

# -------------------------------
# 2️⃣ Evaluate on test data
# -------------------------------
predictions = pipeline_model.transform(test_df)
evaluator = RegressionEvaluator(
    labelCol="purchases",
    predictionCol="prediction",
    metricName="r2"
)
r2 = evaluator.evaluate(predictions)
print(f"✅ Spark Decision Tree R²: {r2:.4f}")

# -------------------------------
# 3️⃣ Infer model signature for MLflow
# -------------------------------
signature = infer_signature(test_df[feature_cols], predictions["prediction"])

# -------------------------------
# 4️⃣ Log model to MLflow
# -------------------------------
mlflow.set_experiment("/Users/ram.katneni@gmail.com/day13_spark_dt")

with mlflow.start_run() as run:
    mlflow.spark.log_model(
        spark_model=pipeline_model,
        artifact_path="spark_dt_model",
        dfs_tmpdir="/Volumes/workspace/ml_models/mlflow_tmp",  # UC temp path
        signature=signature
    )
    mlflow.log_metric("r2", r2)
    run_id = run.info.run_id
    print(f"MLflow Run ID: {run_id}")

# -------------------------------
# 5️⃣ Register model to Unity Catalog
# -------------------------------
model_name = "workspace.ml_models.dt_model_uc"

# Register model in UC
model_uri = f"runs:/{run_id}/spark_dt_model"
mlflow.register_model(model_uri=model_uri, name=model_name)

print(f"Model registered to UC: {model_name}")


#Hyperparameter Tuning for Spark Decision Tree

In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS workspace.ml_models.ecommerce_features (
  views BIGINT,
  purchases BIGINT
)
USING DELTA
""")


In [0]:
spark.sql("""
INSERT INTO workspace.ml_models.ecommerce_features VALUES
(10,1),(20,2),(30,3),(40,5),(50,8),
(60,13),(70,21),(80,34),(90,55),(100,89)
""")


In [0]:
data = spark.table("workspace.ml_models.ecommerce_features")


In [0]:
# ================================
# DAY 13 – Model Comparison & Hyperparameter Tuning
# ================================

# -------------------------------
# 0️⃣ Imports & Environment Setup
# -------------------------------
import os
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline
import mlflow
import mlflow.spark
from mlflow.models.signature import infer_signature
import pandas as pd

# -------------------------------
# 1️⃣ Environment variables
# -------------------------------
# Required for Spark ML on Shared / Serverless clusters
os.environ["SPARKML_TEMP_DFS_PATH"] = "/Volumes/workspace/ml_models/mlflow_tmp"

# MLflow experiment (USER path – NOT UC)
experiment_name = "/Users/ram.katneni@gmail.com/day13_purchases_regression"

if mlflow.get_experiment_by_name(experiment_name) is None:
    mlflow.create_experiment(experiment_name)

mlflow.set_experiment(experiment_name)

# -------------------------------
# 2️⃣ Load & Prepare Data
# -------------------------------
data = spark.table("workspace.ml_models.ecommerce_features")

feature_cols = ["views"]          # raw feature only
label_col = "purchases"

train_df, test_df = data.randomSplit([0.8, 0.2], seed=42)

# -------------------------------
# 3️⃣ Spark ML Pipeline
# -------------------------------
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

dt = DecisionTreeRegressor(
    featuresCol="features",
    labelCol=label_col
)

pipeline = Pipeline(stages=[assembler, dt])

# -------------------------------
# 4️⃣ Hyperparameter Tuning
# -------------------------------
paramGrid = (
    ParamGridBuilder()
    .addGrid(dt.maxDepth, [2, 5, 10])
    .addGrid(dt.minInstancesPerNode, [1, 5, 10])
    .build()
)

evaluator = RegressionEvaluator(
    labelCol=label_col,
    predictionCol="prediction",
    metricName="rmse"   # RMSE is safer than R²
)

crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=3,
    parallelism=2
)

# -------------------------------
# 5️⃣ Train + MLflow + UC Registration (FIXED)
# -------------------------------
mlflow.end_run()

with mlflow.start_run(run_name="Hyperparameter_Tuning_DT_UC"):

    cv_model = crossval.fit(train_df)
    best_model = cv_model.bestModel
    best_dt = best_model.stages[-1]

    predictions = best_model.transform(test_df)
    rmse = evaluator.evaluate(predictions)

    print(f"Best DT: depth={best_dt.getMaxDepth()}, minInstances={best_dt.getMinInstancesPerNode()}")
    print(f"Test RMSE: {rmse}")

    # -------------------------------
    # 🔑 CREATE INPUT / OUTPUT EXAMPLES
    # -------------------------------
    input_example = (
        train_df
        .select(feature_cols)
        .limit(5)
        .toPandas()
    )

    output_example = (
        best_model
        .transform(train_df.limit(5))
        .select("prediction")
        .toPandas()
    )

    signature = infer_signature(input_example, output_example)

    # Log params & metrics
    mlflow.log_param("maxDepth", best_dt.getMaxDepth())
    mlflow.log_param("minInstancesPerNode", best_dt.getMinInstancesPerNode())
    mlflow.log_metric("rmse", rmse)

    # -------------------------------
    # ✅ UC-COMPLIANT MODEL LOGGING
    # -------------------------------
    mlflow.spark.log_model(
        best_model,
        artifact_path="dt_model_spark",
        signature=signature,
        input_example=input_example,
        registered_model_name="workspace.ml_models.dt_model_uc"
    )

    print("✅ Model logged WITH signature and registered in Unity Catalog")

# -------------------------------
# 6️⃣ Feature Importance
# -------------------------------
fi_df = pd.DataFrame(
    zip(feature_cols, best_dt.featureImportances),
    columns=["feature", "importance"]
).sort_values("importance", ascending=False)

print(fi_df)
